#f-scLVM

In this notebook we illustrate how f-scLVM can be used to identify biological processes driving variability between cells.

First, we load some modules and set some directories

In [15]:
import sys
import os
import ast
sys.path.append('./..')
import scipy as SP
import cPickle as pickle
import core.fscLVM as fscLVM
from core.utils import *
import h5py


data_dir = '../../data/'
out_base = './results/'

f-scLVM expects a hdf file containing the normalised, log transformed gene expression data as well as a set of annotations.

In [16]:
dFile = 'Buettneretal.hdf5'
#name of annotation
anno = 'MSigDBdemo'
#number of hidden (unannptated variables)
nHidden = 3
#indices of known covariates
idx_known = []
#number of iterations
nIterations = 500

idxCol=[0,1]
#should the fast option be used (recommende). This only considers genes which are annotated to at least one process.
doFast=True

#specify out_dir
if doFast==False:      
    out_dir = os.path.join(out_base,  dFile.split('.')[0],anno)
else:
    out_dir = os.path.join(out_base,  dFile.split('.')[0],anno+'_fast')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

Next we need to load the relevant data from the hdf5 file.

In [17]:
#min. numer of genes per process
minGenes = 15
dataFile = h5py.File(os.path.join(data_dir, dFile), 'r')

terms = dataFile['terms'][:]#[50:]
pi = dataFile['Pi'][:].T#[:,50:]

Y = dataFile['Yhet'][:].T
#center data
Y-=SP.mean(Y,0)

#exclude small terms
terms = terms[SP.sum(pi>.5,0)>minGenes]
pi = pi[:,SP.sum(pi>.5,0)>minGenes]

#fast option?
if doFast==True:
    idx_genes  = SP.logical_and(SP.sum(pi>.5,1)>0, Y.mean(0)>0.)#SP.any(pi>.5,1)
    Y = Y[:,idx_genes]
    pi = pi[idx_genes,:]

#include hidden variables
terms = SP.hstack([SP.repeat('hidden',nHidden), terms])
pi = SP.hstack([SP.ones((Y.shape[1],nHidden))*.99,pi])


#include known covariates
init_factors = {}
if len(idx_known)>0:
    known_names = dataFile['known_names'][:][idx_known]
    if len(dataFile['Known'][:].shape)>1:
        known = dataFile['Known'][:].T[:,idx_known]
    else:
        known = dataFile['Known'][:][:,SP.newaxis]
    known -= known.mean(0)
    known /= known.std(0)
    terms = SP.hstack([ known_names,terms])
    pi = SP.hstack([SP.ones((Y.shape[1],len(idx_known)))*.5,pi])
    init_factors['Known'] = known             
else:
    known_names = '0'


#specify out_file
out_file = 'resHidden'+str(nHidden)+'_Known__'+'__'.join(known_names)+'.hdf5'
out_name = os.path.join(out_dir, out_file)
print out_name  

terms0=terms.copy()
pi0=pi.copy()




./results/Buettneretal/MSigDBdemo_fast/resHidden3_Known__0.hdf5


Next, we initialise the model and iterate.


In [18]:
reload(fscLVM)
init_factors['iLatent'] = SP.where(terms=='hidden')[0]

#use pre-training to determine initial update order 
Ilabel = preTrain(Y, terms, pi,init_factors) 

#re-order terms
print terms0[Ilabel]
terms = terms0[Ilabel]
pi = pi0[:,Ilabel]

#Set FNR
pi[pi<.1] =1e-3

#initialise model
init={'init_data':sparseFA.CGauss(Y),'Pi':pi,'init_factors':init_factors}
priors = {'Eps': {'priors':[1E-3,1E-3]}}
FA = fscLVM.CSparseFA(components=pi.shape[1], nIterations=nIterations)            
FA.shuffle=True
FA.init(**init) 

#iterate
FA.iterate(forceIterations=True, nIterations=nIterations)

['hidden' 'hidden' 'hidden' 'APICAL_SURFACE' 'P53_PATHWAY' 'G2M_CHECKPOINT'
 'UNFOLDED_PROTEIN_RESPONSE' 'KRAS_SIGNALING_UP' 'ADIPOGENESIS'
 'EPITHELIAL_MESENCHYMAL_TRANSITION' 'DNA_REPAIR' 'MITOTIC_SPINDLE'
 'KRAS_SIGNALING_DN' 'CHOLESTEROL_HOMEOSTASIS' 'BILE_ACID_METABOLISM'
 'UV_RESPONSE_UP' 'INFLAMMATORY_RESPONSE' 'MYC_TARGETS_V2'
 'TGF_BETA_SIGNALING' 'PEROXISOME' 'ANDROGEN_RESPONSE' 'MYC_TARGETS_V1'
 'OXIDATIVE_PHOSPHORYLATION' 'INTERFERON_ALPHA_RESPONSE'
 'FATTY_ACID_METABOLISM' 'APOPTOSIS' 'TNFA_SIGNALING_VIA_NFKB'
 'PI3K_AKT_MTOR_SIGNALING' 'SPERMATOGENESIS' 'PROTEIN_SECRETION'
 'IL6_JAK_STAT3_SIGNALING' 'MTORC1_SIGNALING' 'COMPLEMENT'
 'ALLOGRAFT_REJECTION' 'HYPOXIA' 'E2F_TARGETS' 'HEME_METABOLISM'
 'APICAL_JUNCTION' 'INTERFERON_GAMMA_RESPONSE' 'XENOBIOTIC_METABOLISM'
 'COAGULATION' 'UV_RESPONSE_DN' 'MYOGENESIS' 'ESTROGEN_RESPONSE_LATE'
 'GLYCOLYSIS' 'ESTROGEN_RESPONSE_EARLY' 'IL2_STAT5_SIGNALING']
reconstruction error: 0.741979 lower bound: 0.000000
reconstruction error: 0.8

0.57885554261667116

In [23]:
FA.Pi[:,4]



array([ 0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.99 ,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.99 ,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.99 ,  0.001,  0.99 ,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
        0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,  0.001,
      

We then post-process the result to exclude factors capturing outliers.

In [20]:
MAD = mad(FA.S.E1)
alpha = (MAD>.5)*(1/(FA.Alpha.E1))
idxF = SP.argsort(-alpha)

print terms[idxF]
print alpha[idxF]

#scatter plot of two most important factors
plotFac(FA,idxF[0],idxF[1], terms=terms)

['P53_PATHWAY' 'G2M_CHECKPOINT' 'hidden' 'hidden' 'hidden'
 'PI3K_AKT_MTOR_SIGNALING' 'SPERMATOGENESIS' 'PROTEIN_SECRETION'
 'IL6_JAK_STAT3_SIGNALING' 'MTORC1_SIGNALING' 'COMPLEMENT'
 'ALLOGRAFT_REJECTION' 'HYPOXIA' 'E2F_TARGETS' 'APICAL_JUNCTION'
 'TNFA_SIGNALING_VIA_NFKB' 'INTERFERON_GAMMA_RESPONSE'
 'XENOBIOTIC_METABOLISM' 'COAGULATION' 'UV_RESPONSE_DN' 'MYOGENESIS'
 'ESTROGEN_RESPONSE_LATE' 'GLYCOLYSIS' 'HEME_METABOLISM' 'APOPTOSIS'
 'INTERFERON_ALPHA_RESPONSE' 'ESTROGEN_RESPONSE_EARLY' 'APICAL_SURFACE'
 'UNFOLDED_PROTEIN_RESPONSE' 'KRAS_SIGNALING_UP' 'ADIPOGENESIS'
 'EPITHELIAL_MESENCHYMAL_TRANSITION' 'DNA_REPAIR' 'MITOTIC_SPINDLE'
 'KRAS_SIGNALING_DN' 'FATTY_ACID_METABOLISM' 'CHOLESTEROL_HOMEOSTASIS'
 'UV_RESPONSE_UP' 'INFLAMMATORY_RESPONSE' 'MYC_TARGETS_V2'
 'TGF_BETA_SIGNALING' 'PEROXISOME' 'ANDROGEN_RESPONSE' 'MYC_TARGETS_V1'
 'OXIDATIVE_PHOSPHORYLATION' 'BILE_ACID_METABOLISM' 'IL2_STAT5_SIGNALING']
[ 0.04180916  0.02440418  0.0219986   0.00657431  0.00351484  0.          0.
 

In [ ]:
#save results
out_file = h5py.File(out_name+'.hdf5','w')    
out_file['alphaRaw'] = FA.Alpha.E1
out_file['alpha'] = alpha  
out_file['MAD'] = MAD  
out_file['W'] = FA.W.E1
out_file['Eps'] = FA.Eps.E1
out_file['S'] = FA.S.E1        
out_file['Gamma'] = FA.W.C[:,:,0]
out_file['pi'] = pi
out_file['terms'] = terms
out_file.close()
pickle.dump(FA, open(out_name+'_FA.pickle', 'w'))